In [1]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
import getpass
import pandas as pd

In [2]:
res = requests.get("https://books.toscrape.com/")

In [3]:
soup = BeautifulSoup(res.text , 'html.parser')

In [4]:
# Getting the categories 
soup.find("ul" , {'class':"nav nav-list"})

<ul class="nav nav-list">
<li>
<a href="catalogue/category/books_1/index.html">
                            
                                Books
                            
                        </a>
<ul>
<li>
<a href="catalogue/category/books/travel_2/index.html">
                            
                                Travel
                            
                        </a>
</li>
<li>
<a href="catalogue/category/books/mystery_3/index.html">
                            
                                Mystery
                            
                        </a>
</li>
<li>
<a href="catalogue/category/books/historical-fiction_4/index.html">
                            
                                Historical Fiction
                            
                        </a>
</li>
<li>
<a href="catalogue/category/books/sequential-art_5/index.html">
                            
                                Sequential Art
                            
           

In [5]:
cat_list = soup.find("ul" , {'class':"nav nav-list"}).find_all("li")[1:]

In [6]:
cat_list[:5]

[<li>
 <a href="catalogue/category/books/travel_2/index.html">
                             
                                 Travel
                             
                         </a>
 </li>,
 <li>
 <a href="catalogue/category/books/mystery_3/index.html">
                             
                                 Mystery
                             
                         </a>
 </li>,
 <li>
 <a href="catalogue/category/books/historical-fiction_4/index.html">
                             
                                 Historical Fiction
                             
                         </a>
 </li>,
 <li>
 <a href="catalogue/category/books/sequential-art_5/index.html">
                             
                                 Sequential Art
                             
                         </a>
 </li>,
 <li>
 <a href="catalogue/category/books/classics_6/index.html">
                             
                                 Classics
                 

In [7]:
# Get the name of each category
cat_list[0].find('a').get_text().strip()

'Travel'

In [8]:
# Get the link of each category
cat_list[0].find('a')['href']

'catalogue/category/books/travel_2/index.html'

In [9]:
# Trying to get books in the home page
soup.find_all("article" , {'class':"product_pod"})[:2]

[<article class="product_pod">
 <div class="image_container">
 <a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
 </div>
 <p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>
 <h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
 <div class="product_price">
 <p class="price_color">Â£51.77</p>
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>
 <form>
 <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
 </form>
 </div>
 </article>,
 <article class="product_pod">
 <div class="image_container">
 <a href="catalogue/tipping-the-velvet_999/index.html"><img alt="Tipping the Velvet" class="th

In [10]:
# Get the name of one book then generalize the method
soup.find_all("article" , {'class':"product_pod"})[0].find("h3").find('a')['title']

'A Light in the Attic'

In [11]:
# Get the price of the book
float(soup.find_all("article" , {'class':"product_pod"})[0].find("p" , {'class':'price_color'}).text[2:])

51.77

In [12]:
# Get the rate of the book
soup.find_all("article" , {'class':"product_pod"})[0].find('p')['class'][1]

'Three'

**The rate is not numerical so we should convert it to numerical**

In [13]:
rate_dict = {"One" : 1,
            "Two" : 2,
            "Three" : 3,
            "Four" : 4,
            "Five" : 5}

In [14]:
rate_dict[soup.find_all("article" , {'class':"product_pod"})[0].find('p')['class'][1]]

3

**Everything is good now**

In [15]:
#Trying to find the next button
soup.find("li", {'class':"next"}).find('a')['href']

'catalogue/page-2.html'

**We should make MySQL Connection but there's a security trick first**

In [16]:
passwd = getpass.getpass("Enter the connection password: ")

Enter the connection password: ········


**Creating the database with its name and tables**

In [17]:
DB_Name = "Books"

Table = {}

Table["Category"] = (
    """CREATE TABLE Books.Category (
  CategoryID INT NOT NULL AUTO_INCREMENT,
  CategoryName VARCHAR(45) NULL,
  PRIMARY KEY (CategoryID),
  UNIQUE INDEX CategoryID_UNIQUE (CategoryID ASC) VISIBLE);   """
)

Table['Book'] = (
    """CREATE TABLE Books.Book (
  BookID INT NOT NULL AUTO_INCREMENT,
  BookName VARCHAR(255) NULL,
  BookPrice DECIMAL(5,2) NULL,
  BookRate INT NULL,
  CategoryID INT NULL,
  PRIMARY KEY (BookID),
  INDEX CategoryID_idx (CategoryID ASC) VISIBLE,
  CONSTRAINT CategoryID
    FOREIGN KEY (CategoryID)
    REFERENCES Category (CategoryID)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);
    """
)

In [18]:
conn = mysql.connector.connect(user = "root",
                              password = passwd,
                              host = 'localhost')

In [19]:
cursor = conn.cursor()

**Create the Database**

In [20]:
try:
    cursor.execute(f"CREATE DATABASE {DB_Name}")
except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")

In [21]:
for table_name in Table:
    table_description = Table[table_name]
    try:
        print("Creating table {} ".format(table_name))
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        print(f"Failed creating Table : {err}")

Creating table Category 
Creating table Book 


**We can select the database now to work directly**

In [22]:
conn.database = "Books"

**We should fill Category Table first to have the foreign key for Book Table**

In [23]:
# Check the category first
for i in cat_list:
    print(i.find('a').get_text().strip())
    break

Travel


In [24]:
for i in cat_list:
    try:
        cursor.execute("INSERT INTO Category(CategoryName) VALUES (%s)" , [i.find('a').get_text().strip()])
    except mysql.connector.Error as err:
        print(f'Failed inserting value {err}')

**Everything works fine now , let's commit**

In [25]:
conn.commit()

In [26]:
# We can try a query to make sure everything is working fine
cursor.execute("SELECT * FROM Category")
cursor.fetchall()[:5]

[(1, 'Travel'),
 (2, 'Mystery'),
 (3, 'Historical Fiction'),
 (4, 'Sequential Art'),
 (5, 'Classics')]

**Looks nice, Now it's time to insert the books**

In [27]:
add_book = "INSERT INTO Book(BookName , BookPrice , BookRate , CategoryID) VALUES (%s , %s , %s , %s)"
for i in range(len(cat_list)):
    res_cat = requests.get(f"https://books.toscrape.com/{cat_list[i].find('a')['href']}")
    soup_cat = BeautifulSoup(res_cat.text, 'html.parser')

    # Get the books list inside the current category
    books_list = soup_cat.find_all("article" , {'class':"product_pod"})

    # Iterating throug the books list
    for book in books_list :
        book_name = book.find("h3").find('a')['title']
        book_price = float(book.find("p" , {'class':'price_color'}).text[2:])
        book_rate = rate_dict[book.find('p')['class'][1]]
        cat_name = cat_list[i].find('a').get_text().strip()
        data = [book_name , book_price , book_rate , i + 1]
        cursor.execute(add_book , data)

    #Finding next button
    next_ = soup_cat.find("li" , {"class":"next"})
    while next_:
        next_page = requests.get(f"https://books.toscrape.com/{cat_list[i].find('a')['href'][:-10] + next_.find('a')['href']}")
        next_page_soup = BeautifulSoup(next_page.text , 'html.parser')

        books_list = next_page_soup.find_all("article" , {'class':"product_pod"})

        # Iterating throug the books list
        for book in books_list :
            book_name = book.find("h3").find('a')['title']
            book_price = float(book.find("p" , {'class':'price_color'}).text[2:])
            book_rate = rate_dict[book.find('p')['class'][1]]
            cat_name = cat_list[i].find('a').get_text().strip()
            data = [book_name , book_price , book_rate , i + 1]
            cursor.execute(add_book , data)

        #Asking if there's next button
        next_ = next_page_soup.find("li" , {"class":"next"})
        
conn.commit()

**Without an error, COOL, NOW Let's test SELECT query**

In [28]:
cursor.execute("SELECT * FROM Book")
cursor.fetchall()[:5]

[(1, "It's Only the Himalayas", Decimal('45.17'), 2, 1),
 (2,
  'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  Decimal('49.43'),
  4,
  1),
 (3,
  'See America: A Celebration of Our National Parks & Treasured Sites',
  Decimal('48.87'),
  3,
  1),
 (4,
  'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  Decimal('36.94'),
  2,
  1),
 (5, 'Under the Tuscan Sun', Decimal('37.33'), 3, 1)]

**Awesome!!**<br>
**Let's try join query**

In [29]:
cursor.execute(
    """SELECT * FROM Book
    INNER JOIN Category
    ON Book.CategoryID = Category.CategoryID
    """
)

In [30]:
cursor.fetchall()[:5]

[(1, "It's Only the Himalayas", Decimal('45.17'), 2, 1, 1, 'Travel'),
 (2,
  'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  Decimal('49.43'),
  4,
  1,
  1,
  'Travel'),
 (3,
  'See America: A Celebration of Our National Parks & Treasured Sites',
  Decimal('48.87'),
  3,
  1,
  1,
  'Travel'),
 (4,
  'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  Decimal('36.94'),
  2,
  1,
  1,
  'Travel'),
 (5, 'Under the Tuscan Sun', Decimal('37.33'), 3, 1, 1, 'Travel')]

**Very good**<br>
**Let's use pandas**

In [31]:
select_query = """SELECT BookName , BookPrice , BookRate, CategoryName 
FROM Book
INNER JOIN Category
ON Book.CategoryID = Category.CategoryID
"""

In [32]:
df = pd.read_sql(select_query , conn)

In [33]:
df.head(5)

,BookName,BookPrice,BookRate,CategoryName
0,It's Only the Himalayas,45.17,2,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,Travel
2,See America: A Celebration of Our National Par...,48.87,3,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,Travel
4,Under the Tuscan Sun,37.33,3,Travel


**Awesome!!**

In [34]:
conn.close()